In [203]:
import pandas as pd
import numpy as np

In [229]:
people_df = pd.read_csv(r"C:\Users\19692\Downloads\CSE560-Project\sampled_data\principal_sampled.tsv", sep="\t")
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200453 entries, 0 to 1200452
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   tconst      1200453 non-null  object
 1   ordering    1200453 non-null  int64 
 2   nconst      1200453 non-null  object
 3   category    1200453 non-null  object
 4   job         1200453 non-null  object
 5   characters  1200453 non-null  object
dtypes: int64(1), object(5)
memory usage: 55.0+ MB


In [230]:
people_df1 = pd.read_csv(r"C:\Users\19692\Downloads\CSE560-Project\sampled_data\people_sampled.tsv", sep="\t")
# drop na rows for not null columns
people_df1[["nconst", "primaryName","primaryProfession"]].dropna()
people_df1 = people_df1[people_df1["primaryName"].isna()==False]
people_df1 = people_df1[people_df1["primaryProfession"].isna()==False]
nconst_set = set(people_df1['nconst'])

In [231]:
people_df = people_df[people_df['nconst'].isin(nconst_set)]

## Process for Casts

In [241]:
casts_df = people_df[people_df['characters']!='\\N']
casts_df = casts_df[['tconst','nconst','characters']]

In [242]:
# process characters column
# ["superman, DC"] --> superman/DC
def clean_characters(val):
    if isinstance(val, str):
        return val.strip("[]").replace('"', '').replace("'", '').replace(',', '/').replace(';', '/').replace('\n', '\\n').strip()
    return val

casts_df['characters'] = casts_df['characters'].apply(clean_characters)


In [243]:
# drop duplicate
casts_df = casts_df.drop_duplicates()

## Process for Other Professionals

In [212]:
crew_df = people_df[people_df['characters']=='\\N']
crew_df = crew_df[['tconst','nconst','job','category']]


In [213]:
# get job people
crew_df[['tconst','nconst','job','category']].drop_duplicates().reset_index(drop=True)
crew_df['job'] = crew_df['job'].str.replace(',', '/', regex=False).str.replace(';', '/', regex=False)
crew_df['category'] = crew_df['category'].str.replace(',', '/', regex=False).str.replace(';', '/', regex=False)

In [214]:
crew_df['job'] = crew_df['job'].replace(r'\N','')
crew_df['category'] = crew_df['category'].replace(r'\N','')

In [215]:
crew_df = crew_df[
    crew_df['job'].notna() &
    (crew_df['job'].str.strip() != '')
]


## Connect to PostgreSQL

In [244]:
import psycopg2
import io

In [245]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres.rnxnnqsyjcircloxrryq",
    password="cse560courseproject",
    host="aws-0-us-east-1.pooler.supabase.com",
    port=6543
)
cur = conn.cursor()

In [253]:
conn.rollback()

## Upload Casts Relation

In [254]:
buffer = io.StringIO()
casts_df.to_csv(buffer, index=False, header=False)
buffer.seek(0)

0

In [255]:
cur.copy_expert(
    'COPY "Cast"(tconst, nconst, character) FROM STDIN WITH CSV',
    buffer
)
conn.commit()


np.False_

## Upload Principal

In [219]:
# Step 1: Create a CSV buffer from the DataFrame
buffer = io.StringIO()
crew_df.to_csv(buffer, index=False, header=False)
buffer.seek(0)


0

In [220]:
# upload data to job_category relation
cur.copy_from(buffer, 'crew', sep=',', null='')
conn.commit()